### Association rules by deli
#### Department - deli

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [5]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [6]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="deli"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [10]:
grouped_data

,order_id,product_name
17,1983,Roasted Turkey Breast
18,1983,Organic Roasted Turkey Breast
19,1983,Thick & Chunky Salsa
20,1983,Organic Original Hommus
132,10362,Organic Three Grain Tempeh
133,10362,Organic Extra Firm Tofu
134,10362,Original Tofurky Deli Slices
135,10362,Ground Sausage Style Veggie Protein
232,17829,Fakin' Bacon Organic Smoky Tempeh Strips
233,17829,Organic Roasted Sliced Chicken Breast


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
table1

product_name,25% Less Fat Pepperoni,5 Grain Tempeh,6 Piece with 10 fl oz Spring Water Uploaded Chicken Dunks,8 Piece Fried Chicken,8 Piece Grilled Chicken,"8\"" Pepperoni Pizza",98% Fat Free Premium Ham With Natural Juices,98% Fat Free Premium Ham in Natural Juices,"AFC Sushi Crunchy California Roll Special Prepared In Store, Ready To Eat","AFC Sushi Spicy Salmon Roll Prepared In Store, Ready To Eat",...,Wrap Delight,Yuba Tofu Skin,Za'atar Veggie Burger,Zen Salad,Zesty Garlic Aiolo Mayo Alternative,Zesty Guacamole Dip,Zesty Lemon Hommus,Zesty Lemon Hummus,Zesty Sriracha Carrot Hummus,Zucchini Soup
order_id,,,,,,,,,,,,,,,,,,,,,
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#relace NAN with 0
table1=table1.fillna(0)

In [14]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1
basket_sets = table1.applymap(encode_units)

In [15]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [16]:
frequent_itemsets

,support,itemsets
0,0.012609,[Applewood Smoked Turkey Breast]
1,0.020673,[Avocado Roll]
2,0.021313,[BBQ Chopped Salad]
3,0.011585,[Bacon Style Veggie Protein Strips]
4,0.011585,[Black Bean Salad]
5,0.022849,[Chicken Noodle Soup]
6,0.012737,[Chorizo Seitan]
7,0.018689,[Classic Chicken Salad]
8,0.026562,[Classic Hummus]
9,0.011457,[Classic Hummus Family Size]


In [17]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

In [18]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Roasted Turkey Breast),(Original Hummus),0.095814,0.184780,0.019457,0.203073,1.098999,0.001753,1.022954
1,(Original Hummus),(Organic Roasted Turkey Breast),0.184780,0.095814,0.019457,0.105300,1.098999,0.001753,1.010602
2,(Organic Extra Firm Tofu),(Original Hummus),0.074565,0.184780,0.016065,0.215451,1.165986,0.002287,1.039094
3,(Original Hummus),(Organic Extra Firm Tofu),0.184780,0.074565,0.016065,0.086941,1.165986,0.002287,1.013555
4,(Organic Roasted Turkey Breast),(Organic Roasted Sliced Chicken Breast),0.095814,0.065156,0.015553,0.162325,2.491317,0.009310,1.115998
5,(Organic Roasted Sliced Chicken Breast),(Organic Roasted Turkey Breast),0.065156,0.095814,0.015553,0.238703,2.491317,0.009310,1.187692
6,(Roasted Turkey Breast),(Original Hummus),0.061380,0.184780,0.013185,0.214807,1.162503,0.001843,1.038242
7,(Original Hummus),(Roasted Turkey Breast),0.184780,0.061380,0.013185,0.071354,1.162503,0.001843,1.010741
8,(Roasted Turkey Breast),(Uncured Genoa Salami),0.061380,0.182028,0.018177,0.296142,1.626906,0.007004,1.162127
9,(Uncured Genoa Salami),(Roasted Turkey Breast),0.182028,0.061380,0.018177,0.099859,1.626906,0.007004,1.042748


In [19]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Roasted Turkey Breast),(Original Hummus),0.095814,0.184780,0.019457,0.203073,1.098999,0.001753,1.022954
1,(Original Hummus),(Organic Roasted Turkey Breast),0.184780,0.095814,0.019457,0.105300,1.098999,0.001753,1.010602
2,(Organic Extra Firm Tofu),(Original Hummus),0.074565,0.184780,0.016065,0.215451,1.165986,0.002287,1.039094
3,(Original Hummus),(Organic Extra Firm Tofu),0.184780,0.074565,0.016065,0.086941,1.165986,0.002287,1.013555
4,(Organic Roasted Turkey Breast),(Organic Roasted Sliced Chicken Breast),0.095814,0.065156,0.015553,0.162325,2.491317,0.009310,1.115998
5,(Organic Roasted Sliced Chicken Breast),(Organic Roasted Turkey Breast),0.065156,0.095814,0.015553,0.238703,2.491317,0.009310,1.187692
6,(Roasted Turkey Breast),(Original Hummus),0.061380,0.184780,0.013185,0.214807,1.162503,0.001843,1.038242
7,(Original Hummus),(Roasted Turkey Breast),0.184780,0.061380,0.013185,0.071354,1.162503,0.001843,1.010741
8,(Roasted Turkey Breast),(Uncured Genoa Salami),0.061380,0.182028,0.018177,0.296142,1.626906,0.007004,1.162127
9,(Uncured Genoa Salami),(Roasted Turkey Breast),0.182028,0.061380,0.018177,0.099859,1.626906,0.007004,1.042748


In [20]:
final_result.to_csv("Department_deliApriori.csv",sep=',')

In [21]:
test1=pd.read_csv("Department_deliApriori.csv")

In [22]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [23]:
test1=test1.replace({"u'": ''}, regex=True)

In [24]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [25]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [26]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [27]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Organic Roasted Turkey Breast,Original Hummus,0.095814,0.184780,0.019457,0.203073,1.098999,0.001753,1.022954
1,Original Hummus,Organic Roasted Turkey Breast,0.184780,0.095814,0.019457,0.105300,1.098999,0.001753,1.010602
2,Organic Extra Firm Tof,Original Hummus,0.074565,0.184780,0.016065,0.215451,1.165986,0.002287,1.039094
3,Original Hummus,Organic Extra Firm Tof,0.184780,0.074565,0.016065,0.086941,1.165986,0.002287,1.013555
4,Organic Roasted Turkey Breast,Organic Roasted Sliced Chicken Breast,0.095814,0.065156,0.015553,0.162325,2.491317,0.009310,1.115998
5,Organic Roasted Sliced Chicken Breast,Organic Roasted Turkey Breast,0.065156,0.095814,0.015553,0.238703,2.491317,0.009310,1.187692
6,Roasted Turkey Breast,Original Hummus,0.061380,0.184780,0.013185,0.214807,1.162503,0.001843,1.038242
7,Original Hummus,Roasted Turkey Breast,0.184780,0.061380,0.013185,0.071354,1.162503,0.001843,1.010741
8,Roasted Turkey Breast,Uncured Genoa Salami,0.061380,0.182028,0.018177,0.296142,1.626906,0.007004,1.162127
9,Uncured Genoa Salami,Roasted Turkey Breast,0.182028,0.061380,0.018177,0.099859,1.626906,0.007004,1.042748


In [31]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_DeliApriori.csv")

In [29]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Organic Roasted Turkey Breast,Original Hummus,0.095814,0.184780,0.019457,0.203073,1.098999,0.001753,1.022954
1,Original Hummus,Organic Roasted Turkey Breast,0.184780,0.095814,0.019457,0.105300,1.098999,0.001753,1.010602
2,Organic Extra Firm Tof,Original Hummus,0.074565,0.184780,0.016065,0.215451,1.165986,0.002287,1.039094
3,Original Hummus,Organic Extra Firm Tof,0.184780,0.074565,0.016065,0.086941,1.165986,0.002287,1.013555
4,Organic Roasted Turkey Breast,Organic Roasted Sliced Chicken Breast,0.095814,0.065156,0.015553,0.162325,2.491317,0.009310,1.115998
5,Organic Roasted Sliced Chicken Breast,Organic Roasted Turkey Breast,0.065156,0.095814,0.015553,0.238703,2.491317,0.009310,1.187692
6,Roasted Turkey Breast,Original Hummus,0.061380,0.184780,0.013185,0.214807,1.162503,0.001843,1.038242
7,Original Hummus,Roasted Turkey Breast,0.184780,0.061380,0.013185,0.071354,1.162503,0.001843,1.010741
8,Roasted Turkey Breast,Uncured Genoa Salami,0.061380,0.182028,0.018177,0.296142,1.626906,0.007004,1.162127
9,Uncured Genoa Salami,Roasted Turkey Breast,0.182028,0.061380,0.018177,0.099859,1.626906,0.007004,1.042748


In [30]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()